# 1. Подготовка датасета

На данном этапе производится загрузка и предварительная обработка датасетов для задачи детекции фишинговых email сообщений. Используются два датасета:
- **Nazario Phishing Corpus** (2015-2024) - коллекция фишинговых писем
- **Enron Email Dataset** - коллекция легитимных писем

**Важно**: Парсинг email, извлечение признаков и другие операции выполняются через модули системы (`src/email_parser.py`, `src/feature_extractor.py` и т.д.). В данном блокноте выполняется только загрузка и подготовка сырых данных.




In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import sys
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# Добавляем путь к src для импорта модулей
sys.path.append(str(Path('../src').resolve()))

# Настройка путей
BASE_DIR = Path('../')
DATA_RAW = BASE_DIR / 'data' / 'raw'
DATA_PROCESSED = BASE_DIR / 'data' / 'processed'

print(f"Рабочая директория: {BASE_DIR.resolve()}")
print(f"Исходные данные: {DATA_RAW.resolve()}")
print(f"Обработанные данные: {DATA_PROCESSED.resolve()}")


Рабочая директория: C:\Users\huzch\Documents\project\eml-phishing-analyzer
Исходные данные: C:\Users\huzch\Documents\project\eml-phishing-analyzer\data\raw
Обработанные данные: C:\Users\huzch\Documents\project\eml-phishing-analyzer\data\processed


## 2. Загрузка Nazario Phishing Corpus

Nazario Phishing Corpus представляет из себя отдельные файлы с письмами за период 2015-2024. Файлы представлены в текстовом формате, письма начинаются с "From " и разделены пустой строкой.

In [2]:
def split_nazario_file(file_path: Path) -> list:
    """
    Разделяет файл Nazario на отдельные email сообщения.
    
    Args:
        file_path: Путь к файлу
        
    Returns:
        list: Список строк с содержимым каждого email
    """
    emails = []
    current_email = []
    
    with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
        for line in f:
            # Проверяем, начинается ли строка с "From " (начало нового письма)
            if line.startswith('From ') and len(line) > 20 and '@' in line:
                if current_email:
                    emails.append(''.join(current_email))
                current_email = [line]
            else:
                current_email.append(line)
        
        # Добавляем последнее письмо
        if current_email:
            emails.append(''.join(current_email))
    
    return emails


# Загрузка Nazario Phishing Corpus (2015-2024)
nazario_path = DATA_RAW / 'nazario'
nazario_files = sorted(nazario_path.glob('phishing-*.txt'))

print(f"Найдено файлов Nazario: {len(nazario_files)}")
for f in nazario_files:
    print(f"  - {f.name}")

# Объединение всех файлов
all_nazario_emails = []
for file_path in tqdm(nazario_files, desc="Загрузка Nazario файлов"):
    emails = split_nazario_file(file_path)
    all_nazario_emails.extend(emails)
    print(f"  {file_path.name}: {len(emails)} писем")

print(f"\nВсего загружено фишинговых писем: {len(all_nazario_emails)}")


Найдено файлов Nazario: 10
  - phishing-2015.txt
  - phishing-2016.txt
  - phishing-2017.txt
  - phishing-2018.txt
  - phishing-2019.txt
  - phishing-2020.txt
  - phishing-2021.txt
  - phishing-2022.txt
  - phishing-2023.txt
  - phishing-2024.txt


Загрузка Nazario файлов:   0%|          | 0/10 [00:00<?, ?it/s]

Загрузка Nazario файлов:  10%|█         | 1/10 [00:00<00:01,  5.26it/s]

  phishing-2015.txt: 304 писем


Загрузка Nazario файлов:  20%|██        | 2/10 [00:04<00:22,  2.76s/it]

...

  phishing-2020.txt: 158 писем
  phishing-2021.txt: 101 писем
  phishing-2022.txt: 245 писем


Загрузка Nazario файлов:  90%|█████████ | 9/10 [00:14<00:00,  1.04it/s]

  phishing-2023.txt: 419 писем


Загрузка Nazario файлов: 100%|██████████| 10/10 [00:15<00:00,  1.53s/it]

  phishing-2024.txt: 403 писем

Всего загружено фишинговых писем: 2978


Выведем несколько писем для проверки корректной обработки датасета.

In [3]:
emails_nazario_example = all_nazario_emails[:5]
emails_nazario_example

["From MAILER-DAEMON Thu Sep 28 09:57:25 2017\nDate: 28 Sep 2017 09:57:25 -0400\nFrom: Mail System Internal Data <MAILER-DAEMON@monkey.org>\nSubject: DON'T DELETE THIS MESSAGE -- FOLDER INTERNAL DATA\nMessage-ID: <1506607045@monkey.org>\nX-IMAP: 1506607044 0000000307 $Forwarded\nStatus: RO\n\nThis text is part of the internal format of your mail folder, and is not\na real message.  It is created automatically by the mail system software.\nIf deleted, important folder data will be lost, and it will be re-created\nwith the data reset to initial values.\n\n",
 'From 125117@ihp-osb-lngweb5.ihp.iinet.net.au  Fri Jan  2 07:19:00 2015\nReturn-Path: <125117@ihp-osb-lngweb5.ihp.iinet.net.au>\nX-Original-To: jose@login.monkey.org\nDelivered-To: jose@login.monkey.org\nReceived: from forward.b.hostedemail.com (forward.b.hostedemail.com [64.98.36.17])\n\tby l.monkey.org (Postfix) with ESMTP id 180E6F4007\n\tfor <jose@login.monkey.org>; Fri,  2 Jan 2015 07:19:00 -0600 (CST)\nReceived: from smtpin10.

### 2.1 Сохранение Nazario датасета

После загрузки Nazario датасет сохраняется в структурированном формате (CSV) для дальнейшей обработки.


In [4]:
# Сохранение в DataFrame для дальнейшей обработки
print("Создание DataFrame для Nazario датасета...")
df_nazario = pd.DataFrame({
    'email_content': all_nazario_emails,
    'label': 1 # фишинговое письмо
})


Создание DataFrame для Nazario датасета...


In [5]:
df_nazario.head()

,email_content,label
0,From MAILER-DAEMON Thu Sep 28 09:57:25 2017\nD...,1
1,From 125117@ihp-osb-lngweb5.ihp.iinet.net.au ...,1
2,From 125117@ihp-osb-lngweb5.ihp.iinet.net.au ...,1
3,From kenzel2380@suddenlink.net Sat Jan 3 16:...,1
4,From bjones2010@suddenlink.net Tue Jan 6 15:...,1


In [6]:
print(f"Размер датасета Nazario: {len(df_nazario)}")
print(f"Распределение меток: {df_nazario['label'].value_counts().to_dict()}")

Размер датасета Nazario: 2978
Распределение меток: {1: 2978}


In [7]:
# Сохранение Nazario в CSV
print("\nСохранение Nazario датасета в CSV...")
nazario_csv_path = DATA_PROCESSED / 'nazario_raw.csv'
with tqdm(total=1, desc="Сохранение файла", unit="файл", 
          colour='#A23B72', ncols=100, 
          bar_format='{l_bar}{bar}| {elapsed}<{remaining}') as pbar:
    df_nazario.to_csv(nazario_csv_path, index=False)
    pbar.update(1)

print(f"Nazario датасет сохранен: {nazario_csv_path}")


Сохранение Nazario датасета в CSV...


Сохранение файла: 100%|████████████████████████████████████████████████████████████████| 00:03<00:00

Nazario датасет сохранен: ..\data\processed\nazario_raw.csv


## 3. Загрузка Enron Email Dataset

Enron Email Dataset содержит легитимные письма. Файл представлен в формате CSV. После загрузки будет выполнено сравнение размеров с Nazario датасетом и балансировка при необходимости.


In [8]:
# Определение структуры Enron файла
enron_path = DATA_RAW / 'enron' / 'emails.csv'

# Читаем первые строки для понимания структуры
sample_df = pd.read_csv(enron_path).head()
print(f"Столбцы Enron: {sample_df.columns.tolist()}")
print(f"\nПример данных:")
print(sample_df)

# Определяем столбец с содержимым email
email_content_col = 'message'
print(f"\nИспользуется столбец: {email_content_col}")


KeyboardInterrupt: 

In [ ]:
# Загрузка Enron (по частям из-за большого размера)
chunk_size = 10000
enron_data = []

# Загружаем все доступные данные 
for chunk in tqdm(pd.read_csv(enron_path, chunksize=chunk_size), desc="Чтение Enron"):
    # Берем только нужный столбец
    chunk_emails = chunk[[email_content_col]].copy()
    chunk_emails.columns = ['email_content']
    enron_data.append(chunk_emails)

df_enron_raw = pd.concat(enron_data, ignore_index=True)

print(f"\nЗагружено легитимных писем: {len(df_enron_raw)}")


Чтение Enron: 35it [00:24,  1.45it/s]


KeyboardInterrupt: 

In [ ]:
# Уменьшение размера Enron до 2950 строк для балансировки
import random
target_size = 2950
print(f"Уменьшение размера Enron с {len(df_enron_raw)} до {target_size} строк...")
random.seed(42)
df_enron_raw = df_enron_raw.sample(n=target_size, random_state=42).reset_index(drop=True)
print(f"Размер после уменьшения: {len(df_enron_raw)}")


### 3.1 Сохранение Enron датасета

После загрузки Enron датасет сохраняется в структурированном формате (CSV) для дальнейшей обработки.


In [ ]:
# Сохранение в DataFrame для дальнейшей обработки
print("Создание DataFrame для Enron датасета...")
df_enron = pd.DataFrame({
    'email_content': df_enron_raw['email_content'],
    'label': 0  # легитимное письмо
})

In [ ]:
df_enron.head()


In [ ]:
print(f"Размер датасета Enron: {len(df_enron)}")
print(f"Распределение меток: {df_enron['label'].value_counts().to_dict()}")


In [ ]:
# Сохранение Enron в CSV
print("\nСохранение Enron датасета в CSV...")
enron_csv_path = DATA_PROCESSED / 'enron_raw.csv'
with tqdm(total=1, desc="Сохранение файла", unit="файл", 
          colour='#A23B72', ncols=100, 
          bar_format='{l_bar}{bar}| {elapsed}<{remaining}') as pbar:
    df_enron.to_csv(enron_csv_path, index=False)
    pbar.update(1)

print(f"Enron датасет сохранен: {enron_csv_path}")

In [ ]:
# Сравнение размеров датасетов
nazario_size = len(df_nazario)
enron_size = len(df_enron)

print(f"Размер Nazario (phishing): {nazario_size}")
print(f"Размер Enron (legitimate): {enron_size}")
print(f"Разница: {abs(nazario_size - enron_size)}")

# Балансировка: привести оба датасета к минимальному размеру
min_size = min(nazario_size, enron_size)
if nazario_size != enron_size:
    print(f"\nБалансировка датасетов до минимального размера ({min_size})...")
    df_nazario = df_nazario.sample(n=min_size, random_state=42).reset_index(drop=True)
    df_enron = df_enron.sample(n=min_size, random_state=42).reset_index(drop=True)
    print(f"Размер Nazario после балансировки: {len(df_nazario)}")
    print(f"Размер Enron после балансировки: {len(df_enron)}")
else:
    print("\nРазмеры датасетов уже сбалансированы.")

print(f"\nФинальные размеры:")
print(f"  Nazario (phishing): {len(df_nazario)}")
print(f"  Enron (legitimate): {len(df_enron)}")


## 4. Объединение датасетов

Объединение фишинговых и легитимных писем в единый датасет для дальнейшей обработки модулями системы.


In [ ]:
# Объединение Nazario и Enron в общий датафрейм
df = pd.concat([df_nazario, df_enron], ignore_index=True)

### 4.1 Проверка и очистка данных

In [9]:
df = pd.read_csv(DATA_PROCESSED / 'email_dataset.csv')

In [10]:

# Подсчёт количества пустых значений и дублей до очистки
num_na = df['email_content'].isnull().sum()
num_dupes = df.duplicated().sum()
print(f"Пустых email_content: {num_na}")
print(f"Дубликатов: {num_dupes}")

# Очистка: удаляем пустые и дубликаты
df = df.dropna(subset=['email_content']).drop_duplicates().reset_index(drop=True)

print(f"Размер после очистки: {df.shape}")

print("\nФинальное распределение классов:")
print(df['label'].value_counts())

Пустых email_content: 0
Дубликатов: 0
Размер после очистки: (5900, 2)

Финальное распределение классов:
label
1    2950
0    2950
Name: count, dtype: int64


In [11]:
df.head()

,email_content,label
0,From jose@monkey.org Thu Aug 17 11:41:23 2023 ...,1
1,From jose@monkey.org Fri Sep 27 15:46:37 2024 ...,1
2,From jose@monkey.org Mon Apr 20 11:26:14 2020 ...,1
3,From jose@monkey.org Thu Sep 21 19:14:02 2017 ...,1
4,From jose@monkey.org Wed May 3 17:20:00 2017 ...,1


In [12]:
df.tail()

,email_content,label
5895,Message-ID: <17702800.1075856096893.JavaMail.e...,0
5896,Message-ID: <16654602.1075840221753.JavaMail.e...,0
5897,Message-ID: <21859410.1075852282055.JavaMail.e...,0
5898,Message-ID: <11613616.1075849659137.JavaMail.e...,0
5899,Message-ID: <19463108.1075852096010.JavaMail.e...,0


In [13]:
# Сохранение итогового датасета
output_path = DATA_PROCESSED / 'email_dataset.csv'
df.to_csv(output_path, index=False)
print(f"Итоговый датасет сохранён в {output_path}")

Итоговый датасет сохранён в ..\data\processed\email_dataset.csv


In [14]:
# Подсчёт количества пустых значений и дублей до очистки
num_na = df['email_content'].isnull().sum()
num_dupes = df.duplicated().sum()
print(f"Пустых строк: {num_na}")
print(f"Дубликатов: {num_dupes}")


print("Размерность после очистки:", df.shape)


Пустых строк: 0
Дубликатов: 0
Размерность после очистки: (5900, 2)
